In [ ]:
from __future__ import print_function
import spacy
import json
from json import JSONEncoder

#การค้นหา (SUBJECTS, VERB, OBJECTS, BPMN LABEL)
#การแบ่งประโยค (Set Custom Boundaries)
#การค้นหาประธาน (Subject)
#การค้นหาประธานจากคำเชื่อมประโยค (Subject From Conjunction)
#การค้นหาประธานกับคำปฏิเสธ (Subject with Negated)
#การค้นหาคำปฏิเสธ (isNegated (no, not, n’t, never, none))
#การค้นหาคำกริยา (Verb)
#การค้นหาการกระทำ (Object)
#การค้นหาคำแสดงเงื่อนไข (If, then, else, clause)

#Subject
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]

#Direction
SIMPLE_ENTRY_WORDING = ["start", "starts", "begin", "trigger", "triggers"]
SIMPLE_EXIT_WORDING = ["end", "ends", "finish", "complete", "terminate", "terminates"]
SIMPLE_CLAUSE_WORDING = ["with", "when"]
DIRECT_ORDER_WORDING = ["then", "after", "afterwards", "later", "next", "and"]
CUSTOM_ORDER_WORDING = ["after", "upon", "once", "when"]
CONDITION_CLAUSE_WORDING = ["if", "whether", "in case of", "till", "until", "unless"]
IF_AS_REFERENCE_WORDING = ["In case of","In the latter case","In the former case"]
JOIN_IN_CASE_WORDING = ["In all other case","In both case"] 
ELSE_WORDING = ["else", "otherwise", "if", "not"]
APPROPRIATE_WORDING = ["appropriate", "applicable", "optionally", "necessarily"]
REPETITION_WORDING = ["repeat", "continue", "again"]
COUNT_CONTROLLED_REPETITION_WORDING = ["Once", "Twice"]
NOISY_WORDING = ["have", "posses", "belong", "contain", "consist of", "seem", "need"]
NOISY_WORDING_EXIST = ["be","exist"] 
PHRASES_WORDING = ["Moreover", "In addition"]
AUXILIARY_VERB = ["do", "have", "need", "must", "can", "will", "shall", "etc"]

nsubj_list = []
composite_activity_list = []
dobj_list = []
dobj_right_word_list = []
pobj_list = []

#spaCy Model
nlp = spacy.load("en_core_web_sm")

#กิจกรรมเดี่ยว (Single)
#Simple Entry/Exit => <Composite Activity> (start, begin, (end, finish, complete, terminate)) [with, when] <<Activity>>] 
text_simple_entry_exit_1 = "The incident management process start with the detection of a service failure"
text_simple_entry_exit_2 = "This task terminates with the actual delivery of the freight to the customer"

#Signal Activity => <Activity> (start, begin, trigger, (end, finish, complete, terminate)) <Activity>
text_signal_activity_1 = "The reception of relevant claim document triggers The claims creation process the detection of a service failure ends the claims management process"
text_signal_activity_2 = "Detection of service failure ends claims management process"

#Exit point from CA caused by an Activity
# Actor initiating Entry/Exit
exit_point_from_ca_caused_by_activity  = "Service desk starts incident resolution process"

#====
#Task and Activity
#(1) Task => มองเป็น 1 Predicate (1 ภาคแสดง)
text_task_task_1_predicate = "Call the help desk"
#(2) Activity Sentence => มองเป็น 1 Predicate (1 ภาคแสดง)
text_task_activity_1_predicate = "make a request"
#(1) [and] (2) => Composite Activity
text_composite_activity = "Call the help desk and make a request"

#====
#ผู้ร่วมกิจกรรมแบบกลุ่ม (Group) เช่น การใช้คำว่า Both...and
text_group_actor = "Computer Support Officer and Desktop Support Officer solve the service failure"

#ผู้ทำกิจกรรมและถูกสั่งให้ทำโดยกิจกรรม (Inannimate)
text_in_annimate_1 = "The good is returned to the vendor by receipt department"
text_in_annimate_2 = "Receipt department returns the good to the vendor"

#กิจกรรมที่ถูกแยกกันทำโดยอิสระ (Independent)
text_independent = "Receipt department returns the good to the vendor and system sends notification to the purchase department"

#กิจกรรมทั้งหมดที่ถูกทำด้วยคนเดียว (Single) 1 กิจกรรม
text_single_actor_one_actor = "A client calls the help desk"

#กิจกรรมที่ถูกทำแบบเดี่ยว (Single) 2 กิจกรรม
text_single_actor_two_activity = "A client calls the help desk and makes a request"

#(1) Simple Artifact หรือ <Activity> [Artifact]
#ตัวอย่างประโยค <Call> [the help desk]
text_simple_artifact = "Call the help desk"

#(2) Complex Artifact หรือ <Activity> [Artifact] [and, or] [Artifact]
#ตัวอย่างประโยค <Contact customer via> [email] [or] [telephone]
text_complex_artifact = "Contact customer via email or telephone"

#(3) Shared Artifact หรือ <Activity> [and, or] <Activity [Artifact]>
#Two Task and One Artifact (Both...and)
#ตัวอย่างประโยค [Both] <ask> [and] [only] <answer questions [via email]> = > Two Task and One Artifact
text_shared_artifact = "Both ask and only answer questions via email"

#(4) Independent Artifacts หรือ <Activity [Artifact]> [and, or] <Activity [Artifact]>
#ตัวอย่างประโยค <Call [the help desk] [and] <make [a request]>
text_independent_artifacts = "Call the help desk and make a request"

#Sequence 
#(1) Direct Order => <Previous Activity> (then, after, afterwards later, next, and, (!)) <New Activity>
#ตัวอย่างประโยค <A client calls the help desk> (and) <makes a request>
text_direct_order_1 = "A client calls the help desk and makes a request"

# Before/After
# ตัวอย่างประโยค <Receipt department returns the good to the vendor> (after) <system sends notification to the purchase department>
text_direct_order_2 = "Receipt department returns the good to the vendor after system sends notification to the purchase department"

#(2) Custom Order (After) => (after, upon, once when) <Previous Activity>, <New Activity>
#ตัวอย่างประโยค (After) <an accident occurs>, <the administration team receives advice of the accident via telephone>
text_custom_order_1 = "After an accident occurs, the administration team receives advice of the accident via telephone"
#ตัวอย่างประโยค (when) <a PO Manager creates a PO>, <the system initiates the task automatically>
text_custom_order_2 = "when a PO Manager creates a PO, the system initiates the task automatically"
#ตัวอย่างประโยค <the file is referred to legal support>, (once) <judgement is received> 
text_custom_order_3 = "the file is referred to legal support, once judgement is received"
#Noted USE TIME CLAUSE : https://sites.google.com/site/english25596/bth-thi-3-future-time

#XOR (ใช้ template)
#Exclusive OR => (either, only) <alternative_1> [or] <alternative_2, (no(t) <alternative_1>)>
#ตัวอย่างประโยค {{Officer}} (either) <confirms PO> [or] <rejects>
#Noted : "Either" explicitly indicates XOR-split
text_exclusive_or = "Officer either confirms PO or rejects"

#Inclusive OR => <alternative_1> [and, or] <alternative_2> (ใช้ template)
#ตัวอย่างประโยค {{A client}} <gives a feedback on the previous order> [and/or] <makes a new order>
text_inclusive_or = "A client gives a feedback on the previous order or makes a new order"
text_inclusive_and = "A client gives a feedback on the previous order and makes a new order"

#IF (ใช้ template)
# => (if, whether, in case (of), till, until (unless) <Condition>, <Alternative Activity>
#ตัวอย่างประโยค (In case) <the system detects a service failure> , <it automatically raises a new incident record on the solve system>
#ตัวอย่างประโยค (If) <PO is confirmed> , <a route guide needs to be estimated>
#ตัวอย่างประโยค (Unless) <PO is confirmed> , [do not] <prepare guide>
# Conditional Clause Indicating Alternative Flows
# Fragment of Alternative Flow created by (If-Clause) 
text_if_clause_1 = "In case the system detects a service failure, it automatically raises a new incident record on the solve system" 
text_if_clause_2 = "If PO is confirmed, a route guide needs to be estimated" 
text_if_clause_3 = "Unless PO is confirmed, do not prepare guide"

#IF as a reference (ใช้ template)
#=> (If, whether, in case <Previous Activity>, in [latter] case [of]), <<new Activity>>
#ตัวอย่างประโยค ({{PO is confirmed}}} [or] {{rejected}}, (If) PO is confirmed, <<a route guide needs to be prepared [and] the trailer usage needs to be estimated>>
#ตัวอย่างประโยค (In case of) <a temporary work around>, <<a solve service request for a long term solution is raised based on the solve problem record>>
#ตัวอย่างประโยค (In the latter case), <<the service request process is triggered which is not in scope of incident management>>
#ตัวอย่างประโยค (In the former case), <<the resolve group checks if the incident is correctly assigned to an appropriate group>> 

text_if_as_reference_1 = "PO is confirmed or rejected, If PO is confirmed, a route guide needs to be prepared and the trailer usage needs to be estimated"
text_if_as_reference_2 = "In case of a temporary work around, a solve service request for a long term solution is raised based on the solve problem record"
text_if_as_reference_3 = "In the latter case, the service request process is triggered which is not in scope of incident management"
text_if_as_reference_4 = "In the former case, the resolve group checks if the incident is correctly assigned to an appropriate group"

#Join IF...IF (ใช้ template)
#=> if, whether, in case [of], till, untill (unless) <Activity_1> [and, or] if, whether, in case [of], till, untill (unless) <Activity_2> , [then] <Activity_3>
# - "and" - AND join
# - "or" - OR join หรือ XOR join (ambiguity - see "or" section abuse)
#ตัวอย่างประโยค If <an incident has been handed-off to a resolver group [and] if <an incident has been handled as outlined above, a solve problem record is created>
#ตัวอย่างประโยค If <the resolution was rejected> [or] <updates were made>, <the resolution is reconsidered>
# OR-join by one complex If-Clause 
text_if_and_if = "If an incident has been handed-off to a resolver group and if an incident has been handled as outlined above, a solve problem record is created"
text_if_or_if = "If the resolution was rejected or updates were made, the resolution is reconsidered"

#join In...case (ใช้ template)
#ตัวอย่างประโยค [In all other case], <an incident record is raised in the solve - problem system>
#ตัวอย่างประโยค [If] <resolution doesn't satisfy>, <reject resolution {or} make updates>
#ตัวอย่างประโยค [In both case] <resolution is reconsidered> 
text_join_in_case_1 = "In all other case, an incident record is raised in the solve - problem system"
text_join_in_case_2 = "If resolution doesn't satisfy, reject resolution or make updates"
text_join_in_case_3 = "In both case resolution is reconsidered"

#False...If (ใช้ template)
#ตัวอย่างประโยค The Resolve Group checks [If the incident is correctly assigned to an appropriate group]
#Decide whether a solve problem record is created
#ตัวอย่างประโยค After communicating the resolution, evaluate [If the resolution was permanent] [or] [If additional action is required]
#ตัวอย่างประโยค [If the service was not restored determine] [If the incident require re-assignment]

text_false_if = "The Resolve Group checks If the incident is correctly assigned to an appropriate group"
text_false_if_solve = "Decide whether a solve problem record is created"
text_false_if_after = "After communicating the resolution, evaluate If the resolution was permanent or If additional action is required"
text_false_if_if = "If the service was not restored determine If the incident require re-assignment"

#Else
# => (else, otherwise, if, not), <Alternative Activity> 
#ตัวอย่างประโยค (If it is urgent), <the employee would "hand deliver" the form>, (Otherwise) <it would go via internal mail>
#ตัวอย่างประโยค (Otherwise), <send via email>
# Alternative activity indicated by adverb
text_else_if_otherwise = "If it is urgent, the employee would hand deliver the form, Otherwise it would go via internal mail"
text_else_otherwise = "Otherwise, send via email"

#Option หรือ Alternatively => <<Actor>> [have] (provide, etc) [[option]] ((alternative)) <<<Alternative Activity>>>
#ตัวอย่างประโยค <<Client>> [have] [[an option]] ((to)) <<<notify the incident management team of the failure>>> 
#ตัวอย่างประโยค <<Fiet ORG>> [privide] [[an option]] ((to)) <<<access solve system>>>
text_option = "Client have an option to notify the incident management team of the failure"
text_alternatively = "Fiet ORG privide an option to access solve system"

#Can, may => <Actor> (can, may) <Task>
#ตัวอย่างประโยค <Client> (may) <detect a service failure>
text_may = "Client may detect a service failure"

#Appropriate
#[If] [[it is]] (appropriate, applicable, optionally, not necessarily) <Alternative Action>
#ตัวอย่างประโยค [If] (applicable), <an RVA needs to be recorded in the Meridians Mai IVR System> (Optionally) <Service desk team raise a new incident record> 
## Explicit Optionality by adverb
text_appropriate = "If applicable, an RVA needs to be recorded in the Meridians Mai IVR System Optionally Service desk team raise a new incident record"

#Repetition (ใช้เทมเพลต)
#Pattern 1 Ambiguous Repetition
#Pattern 1.1 Repeat, Continue => (repeat, continue) <Activity> 
#ตัวอย่างประโยค Repeat the assignment
text_repeat = "Repeat the assignment"

#Pattern 1.2 Again => <Activity> (again)
#ตัวอย่างประโยค <The incident is assigned and diagnosed> [again]
text_again = "The incident is assigned and diagnosed again"

#Pattern 1.3 Re-work => re[-]<verb>
#ตัวอย่างประโยค Re-assign the incident
text_re_assign = "Re-assign the incident"

#Pattern 2 Controlled Repetition (ใช้เทมเพลต)
#Pattern 2.1 Condition-Controlled Repetition => (If, whether, in case[of], till, until (unless)) <condition> <<Ambiguous Repetition>>
#ตัวอย่างประโยค (If) <PO was not approved>, <<repeat modifications>>
#ตัวอย่างประโยค (If) <request is not correct>, <<it goes through the process again>>
#ตัวอย่างประโยค (If) <the incident is not correctly assigned>, <<re-assign the incident>>
text_controlled_repetition_1 = "If PO was not approved, repeat modifications"
text_controlled_repetition_2 = "If request is not correct, it goes through the process again"
text_controlled_repetition_3 = "If the incident is not correctly assigned, re-assign the incident"

#Pattern 2.2 Count-Controlled Repetition => <Activity>(Once, Twice)[CD times]
#ตัวอย่างประโยค Print the application from twice
text_count_controlled_repetition = "Print the application from twice"

#Pattern 2.3 Collection controlled repetition
#ตัวอย่างประโยค Each of the documents has to be reviewed
# Activity : Review Document
# Condition : Until all documents are reviewed

#Pattern 1 "Noisy" sentences and clauses
#Pattern 1.1 stative verb = > <Actor> (be,exist), {{have, posses, belong, contain, consist of, seem, need}}
#ตัวอย่างประโยค The default PO Manager is Carmine Marino. A PO contains information about the client's company
text_noisy = "The default PO Manager is Carmine Marino. A PO contains information about the client's company"

#Pattern 1.2 "Dummy Subject-wise Object"
#ตัวอย่างประโยค [It is possible] (that) only one of these or even none of these appointments is made.
# Activity in a subordinate clause of a "noisy" main clause
text_dummy_subject_wise = "It is possible that only one of these or even none of these appointments is made."

#Pattern 1.3 (Subordinate Clauses)
#ตัวอย่างประโยค (As they are in control of the solve system) the raise a new incident record
#ตัวอย่างประโยค (As they are in control of system they raise new record)
text_subordidinate_clause_1 = "As they are in control of the solve system the raise a new incident record"
text_subordidinate_clause_2 = "As they are in control of system they raise new record"

#Pattern 2 "Noisy" words and phrases
#Pattern 2.1 Adverbs & Adjectives
#ตัวอย่างประโยค This results in a confirmation of seventy 1 respectively. [Moreover], the PO is confirmed within 3 days [In addition], the customer notification is issued also, some to final solution, close the solve record, independent from the type of record 
text_adverbs_adjectives = "This results in a confirmation of seventy 1 respectively. Moreover, the PO is confirmed within 3 days In addition, the customer notification is issued also, some to final solution, close the solve record, independent from the type of record"

#Pattern 2.2 Auxiliary Verbs => [Actor] (be), {{do, have, need, must, can, will, shall, etc}} <<Activity>>
#ตัวอย่างประโยค Whiteboard needs to be completed -> "cleared Activity : Complete whiteboard” If it is urgent, [the employee] (would) "hard deliver" the form, (otherwise) it would go via internal mail


#TEST
text = "Officer either confirms PO or rejects"
doc = nlp(text)

#==================================================================================
#Get Noun Phrase
noun_chunks_list = []
for np in doc.noun_chunks: 
    noun_chunks_list.append(str(np))

#==================================================================================
def getAllSubsLefts(v):
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.dep_ is not "det"]
    #กรณีหาประธานไม่ได้ เจอ Activity ที่เป็น Compound
    getCompound(v)
    return subs

def getCompound(v):
    compound = [tok for tok in v.lefts if tok.dep_ == "compound"]
    composite_activity_list.append(compound)

def getObjectRights(v):
    object_sentence = [tok for tok in v.rights]
    return object_sentence

def getMultipleVerbRights(v):
    multiple_verb_sentence = [tok for tok in v.rights if tok.dep_ == "conj"]
    return multiple_verb_sentence

verbs = [tok for tok in doc if tok.pos_ == "VERB" and tok.dep_ != "aux"]
for v in verbs:
    subs = getAllSubsLefts(v)
    dobj_right_word_list.append(getObjectRights(v))
#==================================================================================
#Get Noun Phrase relation with verb
index = 0
nounIndices = []
for token in doc:
    if token.pos_ == 'NOUN':
        nounIndices.append(index)
    index = index + 1

for idxValue in nounIndices:
    doc = nlp(text)
    span = doc[doc[idxValue].left_edge.i : doc[idxValue].right_edge.i+1]
    span.merge()

    for token in doc:
        if token.dep_ in SUBJECTS:
            nsubj_list.append(token.text)
        elif token.dep_ == 'dobj' or token.dep_ == 'pobj':
            dobj_list.append(token.text)
        elif token.dep_ == 'pobj' or token.pos_ == "PRON":
            pobj_list.append(token.text)

#Get Noun Phrase
print("nsubj_list :", nsubj_list)
print("dobj_list :", dobj_list)
print("pobj_list :", pobj_list)

if not nsubj_list:
    print("กรณีหาประธานไม่ได้ เจอ Activity ที่เป็น Compound")
    all_subject = str(composite_activity_list[0]).replace('[','').replace(']','')
else:
    print("หาประธานได้", nsubj_list, noun_chunks_list)
    all_subject = " ".join(set(nsubj_list).intersection(noun_chunks_list))
    # กรณีไม่เจอ Subject เลย
    if len(all_subject) == 0:
        all_subject = str(noun_chunks_list[0])
        print(all_subject)

all_verb = str(verbs[0])

if not dobj_list:
    print("ไม่มี object")
    all_object = str(dobj_right_word_list[0]).replace('[','').replace(']','')
else:
    print("มี object จาก Noun Phrase")
    all_object = " ".join(set(dobj_list).intersection(noun_chunks_list))

all_clause = ""

#====================================JSON=============================================
try:
    from types import SimpleNamespace as Namespace
except ImportError:
    # Python 2.x fallback
    from argparse import Namespace

class ActiveSentence:
    def __init__(self, nsubj, verb, dobj, clause):
        self.nsubj, self.verb, self.dobj, self.clause = nsubj, verb, dobj, clause

class PassiveSentence:
    def __init__(self, nsubjpass, verbpass, dobj, clause):
        self.nsubjpass, self.verbpass, self.dobj, self.clause = nsubjpass, verbpass, dobj, clause

class BPMNProcess:
    def __init__(self, sequence, signalWord, activityName, conditionName, participantsName, symbol):
        self.sequence, self.signalWord, self.activityName, self.conditionName, self.participantsName, self.symbol = sequence, signalWord, activityName, conditionName, participantsName, symbol

class Condition:
    def __init__(self, sequence, condition):
        self.sequence, self.condition = sequence, condition

class ActivityOneSentence:
    def __init__(self, sentence, process):
        self.sentence, self.process = sentence, process

class ActivityMoreSentence:
    def __init__(self, activeSentence, passiveSentence, process):
        self.activeSentence, self.passiveSentence, self.process = activeSentence, passiveSentence, process

class BPMNEncoder(JSONEncoder):
        def default(self, o): return o.__dict__

def create_sentence():
    activeSentence = ActiveSentence(all_subject, all_verb, all_object, all_clause) 
    symbol = get_bpmn_symbol(all_verb)
    vo_label = all_verb+" "+all_object
    bpmnProcess = BPMNProcess(1,"",vo_label,"",all_subject, symbol)
    create_all_sentence(activeSentence, None, bpmnProcess)

def get_bpmn_symbol(verb):
    if verb in SIMPLE_ENTRY_WORDING:
        return "START"
    elif verb in SIMPLE_EXIT_WORDING:
        return "END"
    else:
        return "TASK"
    
def create_all_sentence(activeSentence, passiveSentence, bpmnProcess):
    #Have 2 sentence 
    data = []
    createActivity = None
    if activeSentence is None or passiveSentence is None:
        if activeSentence is not None: 
            createActivity = ActivityOneSentence(activeSentence, bpmnProcess)
            data.append(createActivity)
        else:
            createActivity = ActivityOneSentence(passiveSentence, bpmnProcess)
            data.append(createActivity)
    else:
        createActivity = ActivityMoreSentence(activeSentence, passiveSentence, bpmnProcess)
        data.append(createActivity)
        
    bpmnJsonAllData = json.dumps(data, indent=4, cls=BPMNEncoder)
    print(bpmnJsonAllData)

create_sentence()

nsubj_list : ['Officer', 'Officer', 'Officer']
dobj_list : ['PO', 'PO', 'PO']
pobj_list : []
หาประธานได้ ['Officer', 'Officer', 'Officer'] ['Officer', 'PO', 'rejects']
มี object จาก Noun Phrase
[
    {
        "sentence": {
            "nsubj": "Officer",
            "verb": "confirms",
            "dobj": "PO",
            "clause": ""
        },
        "process": {
            "sequence": 1,
            "signalWord": "",
            "activityName": "confirms PO",
            "conditionName": "",
            "participantsName": "Officer",
            "symbol": "TASK"
        }
    }
]
